In [4]:
import pickle
data = pickle.load(open("data/data.plk", "rb"))

In [5]:
len(data)

4351

In [6]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk.corpus import stopwords


wordnet_lemmatizer = WordNetLemmatizer()

In [7]:
def get_wordnet_pos(treebank_tag):

    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

In [8]:
def get_lemmatized_words(sentence):
    _temp = []
    tokens = nltk.word_tokenize(sentence)
    
    for word, token in nltk.pos_tag(tokens):
        if word.strip().lower() not in stopwords.words('english') and word.strip().lower().isalpha() and len(word.strip().lower()) > 1:
            _temp.append(wordnet_lemmatizer.lemmatize(word.strip().lower(), get_wordnet_pos(token)))
    return _temp

In [11]:
[" ".join(get_lemmatized_words(x["název (anglicky)"])) for x in data[:2] if len(x["název (anglicky)"]) > 2 and len(x["katedra"]) > 1]

['salzella declarative language music generation',
 'application illustrative explode view 3d model']

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
names_list = [" ".join(get_lemmatized_words(x["název (anglicky)"])) for x in data if len(x["název (anglicky)"]) > 2 and len(x["katedra"]) > 1]

In [10]:
vectorizer = TfidfVectorizer(use_idf=True, ngram_range=(1,3), min_df=1) # min_df if int then it is absolute value
X = vectorizer.fit_transform(names_list)

In [11]:
X[0]

<1x24759 sparse matrix of type '<class 'numpy.float64'>'
	with 12 stored elements in Compressed Sparse Row format>

In [12]:
print (X[0])

  (0, 18271)	0.309093230645
  (0, 5568)	0.309093230645
  (0, 11815)	0.200783367841
  (0, 14228)	0.229641356648
  (0, 8881)	0.217032853369
  (0, 18272)	0.309093230645
  (0, 5569)	0.309093230645
  (0, 11844)	0.309093230645
  (0, 14233)	0.309093230645
  (0, 18273)	0.309093230645
  (0, 5570)	0.309093230645
  (0, 11845)	0.309093230645


In [13]:
from sklearn.decomposition import TruncatedSVD
lsa = TruncatedSVD(n_components=20, n_iter=100)
lsa.fit(X)

TruncatedSVD(algorithm='randomized', n_components=20, n_iter=100,
       random_state=None, tol=0.0)

**LSA**

Input: X, a matrix where m is the number of documents I have, and n is the number of terms.
Process: I'm going to decompose X into three matricies called U, S, and T. When we do the decomposition, we have to pick a value k, that's how many concepts we are going to keep.
$$X \approx USV^{T}$$

U will be a m x k matrix. The rows will be documents and the columns will be 'concepts'

S will be a k x k diagnal matrix. The elements will be the amount of variation captured from each concept.

V will be a n x k (mind the transpose) matrix. The rows will be terms and the columns will be conepts.

In [14]:
lsa.components_[0].shape

(24759,)

In [15]:
terms = vectorizer.get_feature_names()
for i, comp in enumerate(lsa.components_): 
    termsInComp = zip (terms,comp)
    sortedTerms =  sorted(termsInComp, key=lambda x: x[1], reverse=True) [:10]
    print("Concept %d:" % i )
    for term in sortedTerms:
        print(term[0])
    print (" ")

Concept 0:
system
information
information system
management
management system
web
content management
content management system
content
project
 
Concept 1:
management
web
application
management system
content
content management
content management system
project
web application
project management
 
Concept 2:
management
management system
content management
content management system
content
system
project management
document management
project
document
 
Concept 3:
network
implementation
data
computer
computer network
neural
neural network
design
user
user interface
 
Concept 4:
interface
user
user interface
graphical
graphical user
graphical user interface
design
system
web interface
implementation
 
Concept 5:
project
application
project management
management
mobile
software
support
tool
mobile application
android
 
Concept 6:
portal
network
project
project management
management
software
interface
information
web
computer network
 
Concept 7:
implementation
portal
data
project
tool
des

In [16]:
lsa = TruncatedSVD(n_components=2, n_iter=100)
lsa.fit(X)
terms = vectorizer.get_feature_names()
for i, comp in enumerate(lsa.components_): 
    termsInComp = zip (terms,comp)
    sortedTerms =  sorted(termsInComp, key=lambda x: x[1], reverse=True) [:10]
    print("Concept %d:" % i )
    for term in sortedTerms:
        print(term[0])
    print (" ")

Concept 0:
system
information
information system
management
management system
web
content management
content management system
content
project
 
Concept 1:
management
web
application
management system
content
content management
content management system
project
web application
project management
 


In [18]:
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer

def calculate_lsa(n_components, names_list, vectorizer):
    X = vectorizer.fit_transform(names_list)

    lsa = TruncatedSVD(n_components=20, n_iter=100)
    lsa.fit(X)

    terms = vectorizer.get_feature_names()
    for i, comp in enumerate(lsa.components_): 
        termsInComp = zip (terms,comp)
        sortedTerms =  sorted(termsInComp, key=lambda x: x[1], reverse=True) [:10]
        print("Concept %d:" % i )
        for term in sortedTerms:
            print(term[0])
        print (" ")

In [19]:
names_list = [" ".join(get_lemmatized_words(x["název (anglicky)"])) for x in data if len(x["název (anglicky)"]) > 2 and len(x["katedra"]) > 1]
vectorizer = TfidfVectorizer(use_idf=True, ngram_range=(1,3), min_df=1) # min_df if int then it is absolute value

calculate_lsa(2, names_list, vectorizer)

Concept 0:
system
information
information system
management
management system
web
content management
content management system
content
project
 
Concept 1:
management
web
application
management system
content
content management
content management system
project
web application
project management
 
Concept 2:
management
management system
content management
content management system
content
system
project management
document management
project
document
 
Concept 3:
network
implementation
data
computer
computer network
neural
neural network
design
user
user interface
 
Concept 4:
interface
user
user interface
graphical
graphical user
graphical user interface
design
system
web interface
implementation
 
Concept 5:
project
application
project management
management
mobile
software
support
tool
mobile application
android
 
Concept 6:
portal
network
project
project management
management
software
interface
information
web
computer network
 
Concept 7:
implementation
portal
data
project
tool
des

In [ ]:
ß